In [2]:
# ! pip install git+https://github.com/facebookresearch/ImageBind
! pip install soundfile
! pip install librosa
! pip install "imagebind @ git+https://github.com/facebookresearch/ImageBind@c6a47d6dc2b53eced51d398c181d57049ca59286"

  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl (1.0 MB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)


In [2]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
# create triplet dataset
from datasets import load_dataset
import soundfile as sf
import requests
import pandas as pd
# https://huggingface.co/datasets/agkphysics/AudioSet
# dataset = load_dataset("agkphysics/AudioSet", "bal", split="test")
dataset = load_dataset("arrow", split="test", data_files={"test":"C:/Users/Christina/.cache/huggingface/datasets/agkphysics___audio_set/bal/audio_set-test-000**-of-00018.arrow"})#, split="test[0:10]")
# https://huggingface.co/docs/datasets/v2.16.1/en/package_reference/main_classes#datasets.Dataset.filter
dataset_it = dataset.to_iterable_dataset()

data_dir = "C:\\Users\\Christina\\Data\\imagebind\\text-audio-image\\"
animals = ["Bird", "Cat", "Dog", "Horse"]

infos = []
idx = 0
for animal in animals:
    for example in dataset_it.filter(lambda x: animal in x["human_labels"]).take(25):
        # save audiofile
        sf.write(data_dir + "audio\\%03d.wav"%idx, example["audio"]["array"], example["audio"]["sampling_rate"], format="wav")

        # save image
        # https://stackoverflow.com/questions/2068344/how-do-i-get-a-youtube-video-thumbnail-from-the-youtube-api
        url = 'https://img.youtube.com/vi/%s/hqdefault.jpg'%example["video_id"]
        data = requests.get(url).content 
        with open(data_dir + "image\\%03d.jpg"%idx,'wb') as f:
            f.write(data) 

        # save info
        infos.append({"id": idx, "video_id":example["video_id"], "labels":example["human_labels"], "animal": animal})

        idx += 1

pd.DataFrame(infos).to_csv(data_dir + "info.csv", index=False)

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [36]:
# # https://huggingface.co/docs/datasets/loading#slice-splits
# from datasets import load_dataset

# # dataset = load_dataset("agkphysics/AudioSet", "bal", split="test") #, streaming=True
# dataset = load_dataset("arrow", split="test", data_files={"test":"C:/Users/Christina/.cache/huggingface/datasets/agkphysics___audio_set/bal/audio_set-test-000**-of-00018.arrow"})#, split="test[0:10]")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


# Image-Text

In [1]:
from amumo import model as am_model
from amumo import data as am_data
from amumo import utils as am_utils
from amumo import widgets as am_widgets

# load dataset
# dataset = am_data.DiffusionDB_Dataset(path="2m_first_1k", batch_size=100) # data helper for the diffusionDB dataset; for the interactive prototype, we only use a random subset of 100 samples
# all_images, all_prompts = dataset.get_data()
# cache_name = 'diffusiondb_random_100' # path used to cache the results

C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\uma

In [3]:
# Data Helpers
def get_data_helper(dataset, filters=[], method=any):
    all_images, all_prompts = dataset.get_filtered_data(filters, method=method)
    print(len(all_images))

    dataset_name = dataset.name
    if len(filters) > 0:
        dataset_name = dataset_name + '_filter-' + method.__name__ + '_' + '-'.join(filters)
    else:
        dataset_name = dataset_name + '_size-%i'%len(all_images)

    return all_images, all_prompts, dataset_name

# Load Data
data_path = '../../../../../Data/'
# subset of mscoco validation data
dataset_mscoco_val = am_data.MSCOCO_Val_Dataset(path=data_path+'mscoco/validation', batch_size=100) 
mscoco_val_images, mscoco_val_prompts, mscoco_val_dataset_name = get_data_helper(dataset_mscoco_val, filters=[], method=any)
mscoco_val_dataset_name

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
100


'MSCOCO-Val_size-100'

In [4]:
am_widgets.CLIPExplorerWidget(mscoco_val_dataset_name, all_data={"image": mscoco_val_images, "text": mscoco_val_prompts}, models=[am_model.ImageBind_Model(), "CLIP"]) # {"image": all_images, "depth": all_depths}

c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


batch 1 of 1
batch 1 of 1


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('ImageBind…

found cached embeddings for MSCOCO-Val_size-100_CLIP_RN50 image
found cached embeddings for MSCOCO-Val_size-100_CLIP_RN50 text
 False
found cached embeddings for MSCOCO-Val_size-100_ImageBind_huge image
found cached embeddings for MSCOCO-Val_size-100_ImageBind_huge text
found cached embeddings for MSCOCO-Val_size-100_CLIP_RN50 image
found cached embeddings for MSCOCO-Val_size-100_CLIP_RN50 text
found cached embeddings for MSCOCO-Val_size-100_ImageBind_huge image
found cached embeddings for MSCOCO-Val_size-100_ImageBind_huge text
 True
 True


c:\users\christina\repositories\icg\researchstay\amumo\amumo\utils.py:126: ClusterWarning:

The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix



 False
found cached embeddings for MSCOCO-Val_size-100_ImageBind_huge image
found cached embeddings for MSCOCO-Val_size-100_ImageBind_huge text
found cached embeddings for MSCOCO-Val_size-100_ImageBind_huge image
found cached embeddings for MSCOCO-Val_size-100_ImageBind_huge text
 True
found cached embeddings for MSCOCO-Val_size-100_CLIP_RN50 image
found cached embeddings for MSCOCO-Val_size-100_CLIP_RN50 text


In [6]:
am_widgets.CLIPComparerWidget(cache_name, all_images=all_images, all_prompts=all_prompts, models=[am_model.ImageBind_Model(), "CLIP"]) # {"image": all_images, "depth": all_depths}

True [False, False] {'ImageBind': <amumo.model.ImageBind_Model object at 0x000001B878F032E0>, 'CLIP': <amumo.model.CLIPModel object at 0x000001B767795E20>}


c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:940: FutureWarning:

The input object of type 'PngImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'PngImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

c:\users\christina\repositories\icg\researchstay\amumo\amumo\widgets.py:940: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



found cached embeddings for diffusiondb_random_100_ImageBind_huge
found cached embeddings for diffusiondb_random_100_CLIP_RN50


CLIPComparerWidget(children=(HoverWidget(children=(VBox(children=(HTML(value='', layout=Layout(width='300px'))…

# Image-Text-Audio

In [1]:
from amumo import model as am_model
from amumo import data as am_data
from amumo import utils as am_utils
from amumo import widgets as am_widgets
# ! pip install torch
# ! pip install torchaudio
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\Christina\AppData\Roaming\Python\Python39\site-packages\uma

In [2]:


from glob import glob
from PIL import Image
import numpy as np
import io
import torchaudio
import pandas as pd


class AudioType(am_data.DataTypeInterface):
    name = "Audio"

    def __init__(self, data, sample_rate) -> None:
        super().__init__(data)
        self.sample_rate = sample_rate
    
    def getVisItem(self, idx):
        import soundfile as sf
        buffer = io.BytesIO()
        sf.write(buffer, self.data[idx][0], self.sample_rate, format="wav")
        audio_bytes = buffer.getvalue() # wav bytes
        return {"displayType": am_data.DisplayTypes.AUDIO, "value": audio_bytes}



class Triplet_Dataset(am_data.DatasetInterface):
    name='Triplet'

    def __init__(self, path, seed=31415, batch_size=100, sample_rate=16000):
        # create triplet dataset if it does not exist
        super().__init__(path, seed, batch_size)
        # path: path to the triplet dataset
        image_paths = glob(path + "image\\*.jpg", recursive = True)
        audio_paths = glob(path + "audio\\*.wav", recursive = True)

        self.sample_rate = sample_rate
        
        all_images = []
        for image_path in image_paths:
            with open(image_path, "rb") as fopen:
                image = Image.open(fopen).convert("RGB")
                all_images.append(image)

        all_audios = []
        for audio_path in audio_paths:
            waveform, sr = torchaudio.load(audio_path)
            if sample_rate != sr:
                waveform = torchaudio.functional.resample(
                    waveform, orig_freq=sr, new_freq=sample_rate
                )
            all_audios.append(waveform)
        
        self.all_infos = pd.read_csv(path + "info.csv", converters={"labels": lambda x: x.strip("[]").replace("'","").split(", ")})

        # TODO... load on demand with a custom loader
        self.all_images = np.array(all_images)
        self.all_prompts = np.array(self.all_infos["labels"].map(lambda x: ", ".join(x)))
        self.all_audios = np.array(all_audios)
    
    
    def get_data(self):
        
        if self.batch_size is None:
            images = self.MODE1_Type(self.all_images)
            texts = self.MODE2_Type(self.all_prompts)
            audios = AudioType(self.all_audios, self.sample_rate)
        
            return images, texts, audios

        # create a random batch
        batch_idcs = self._get_random_subsample(len(self.all_images))

        images = self.MODE1_Type(self.all_images[batch_idcs])
        texts = self.MODE2_Type(self.all_prompts[batch_idcs])
        audios = AudioType(self.all_audios[batch_idcs], self.sample_rate)
        
        return images, texts, audios
    
        
    def get_filtered_data(self, filter_list, method=any):
        # filter_list: a list of strings that are used for filtering
        # method: any -> any substring given in filter_list is present; all -> all substrings must be contained in the string
        if filter_list is None or len(filter_list) <= 0:
            return self.get_data()

        subset_ids = np.array([i for i in range(len(self.all_prompts)) if method(substring in self.all_prompts[i].lower() for substring in filter_list)])
        if len(subset_ids) <= 0:
            print("no filter matches found")
            return [], [], []
        
        # create a random batch
        batch_idcs = self._get_random_subsample(len(subset_ids))
        subset_ids = subset_ids[batch_idcs]
        
        images = self.MODE1_Type(self.all_images[subset_ids])
        texts = self.MODE2_Type(self.all_prompts[subset_ids])
        audios = AudioType(self.all_audios[subset_ids], self.sample_rate)
        return images, texts, audios


In [5]:
dataset = Triplet_Dataset(path="C:\\Users\\Christina\\Data\\imagebind\\text-audio-image\\", batch_size=100)
all_images, all_prompts, all_audios = dataset.get_data()
print(len(all_images), len(all_prompts), len(all_audios))

from importlib import reload
reload(am_widgets)
my_widget = am_widgets.CLIPExplorerWidget("test_audio", all_data={"text": all_prompts, "audio": all_audios}, models=[am_model.ImageBind_Model()])
my_widget.scatter_widget.select_projection_method.value = "PCA"
my_widget

C:\Users\Christina\AppData\Local\Temp\ipykernel_33712\2901351163.py:55: FutureWarning:

The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

C:\Users\Christina\AppData\Local\Temp\ipykernel_33712\2901351163.py:55: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

C:\Users\Christina\AppData\Local\Temp\ipykernel_33712\2901351163.py:57: FutureWarning:

The input object of type 'Tensor' is an array-

100 100 100
found cached embeddings for test_audio_ImageBind_huge text
found cached embeddings for test_audio_ImageBind_huge audio
 False


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('ImageBind…

99 39 ['text', 'audio'] 1 0
98 38 ['audio'] 0 0
93 39 ['audio'] 0 0
90 43 ['audio'] 0 0
65 65 ['audio'] 0 0
43 83 ['audio'] 0 0
41 91 ['audio'] 0 0
92 86 ['audio'] 0 0
13 91 ['audio'] 0 0
33 64 ['audio'] 0 0
68 57 ['audio'] 0 0
98 73 ['text'] 0 0
87 68 ['text'] 0 0
87 62 ['text'] 0 0
87 51 ['text'] 0 0
87 50 ['text'] 0 0
84 39 ['text'] 0 0
81 32 ['text'] 0 0
72 33 ['text'] 0 0
54 36 ['text'] 0 0
52 36 ['text'] 0 0
46 43 ['text'] 0 0
44 46 ['text'] 0 0
45 46 ['text'] 0 0
46 45 ['text'] 0 0
45 44 ['text'] 0 0
44 44 ['text'] 0 0
54 46 ['text'] 0 0


In [5]:
# dataset = Triplet_Dataset(path="C:\\Users\\Christina\\Data\\imagebind\\text-audio-image\\", batch_size=100)
# all_images, all_prompts, all_audios = dataset.get_data()
# print(len(all_images), len(all_prompts), len(all_audios))

my_widget = CLIPExplorerWidget("test_audio", all_data={"text": all_prompts, "audio": all_audios, "image": all_images}, models=[am_model.ImageBind_Model()]) 
my_widget.scatter_widget.select_projection_method.value = "PCA"
my_widget

found cached embeddings for test_audio_ImageBind_huge text
found cached embeddings for test_audio_ImageBind_huge audio
found cached embeddings for test_audio_ImageBind_huge image
 False


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('ImageBind…

In [21]:
TSNE().fit(np.random.rand(100, 100)).transform(np.random.rand(100, 100))

PartialTSNEEmbedding([[ 8.66231252e-01, -1.80334395e+00],
                      [-6.98845549e-02,  4.86421987e-01],
                      [-1.80532078e+00, -4.17764983e-01],
                      [-1.69716285e+00, -4.86806520e-01],
                      [ 6.34820991e-01,  4.77157939e-01],
                      [-7.02025889e-01,  9.50794310e-01],
                      [ 6.78130001e-01,  8.57959178e-01],
                      [-1.88099454e+00,  8.72435976e-01],
                      [-1.56499882e-02,  1.42629699e-01],
                      [ 2.93348297e-01,  2.69271462e+00],
                      [ 1.79785216e+00, -2.89591829e-01],
                      [ 8.25588826e-01, -3.33093071e-01],
                      [-3.73619880e-01, -8.62589914e-01],
                      [-6.13675611e-01,  1.70157657e+00],
                      [ 1.13072201e+00, -1.31279701e+00],
                      [ 1.03003015e+00,  9.68657049e-01],
                      [-1.19196244e+00, -7.00524880e-01],
              

In [23]:

embeddings, logit_scale = get_embeddings_per_modality(am_model.ImageBind_Model(), "test_audio", {"image": all_images, "text": all_prompts, "audio": all_audios}, batch_size=100)

heatmap = SimilarityHeatmapClusteringWidget(cluster_label_data=all_prompts)
heatmap.embedding = {"image": np.array(embeddings["image"]), "text": np.array(embeddings["text"]), "audio": np.array(embeddings["audio"])}
heatmap


100 100 100
found cached embeddings for test_audio_ImageBind_huge image
found cached embeddings for test_audio_ImageBind_huge text
found cached embeddings for test_audio_ImageBind_huge audio


SimilarityHeatmapClusteringWidget(children=(HBox(children=(Checkbox(value=False, description='Cluster matrix b…

In [23]:
heatmap.value.shape

(200, 200)

In [18]:
similarities_all = get_similarities_all(heatmap.embedding)
len(similarities_all)

300

In [21]:
(similarities_all[np.concatenate([heatmap.idcs + i*heatmap.size for i in range(len(heatmap.modality_labels))], axis=0),:][:,np.concatenate([heatmap.idcs + i*heatmap.size for i in range(len(heatmap.modality_labels))], axis=0)]).shape

(300, 300)

In [3]:

my_widget = am_widgets.CLIPExplorerWidget("test_audio", all_data={"text": all_prompts, "image": all_images, "audio": all_audios }, models=[am_model.ImageBind_Model()]) 
my_widget

C:\Users\Christina\AppData\Local\Temp\ipykernel_30488\3688791554.py:54: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  self.all_images = np.array(all_images)
C:\Users\Christina\AppData\Local\Temp\ipykernel_30488\3688791554.py:54: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.all_images = np.array(all_images)
C:\Users\Christina\AppData\Local\Temp\ipykernel_30488\36

100 100 100


c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\Christina\anaconda3\envs\amumo_test\lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


found cached embeddings for test_audio_ImageBind_huge text
found cached embeddings for test_audio_ImageBind_huge image
found cached embeddings for test_audio_ImageBind_huge audio


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('ImageBind…

 False


# Image-Thermal

In [ ]:
from glob import glob
from PIL import Image
import numpy as np


class ThermalType(am_data.ImageType):
    name = "Thermal"

    def __init__(self, data) -> None:
        super().__init__(data)


class LLVIP_Dataset(am_data.DatasetInterface):
    name='LLVIP'

    def __init__(self, path, seed=31415, batch_size = 100):
        # download dataset: https://bupt-ai-cz.github.io/LLVIP/
        super().__init__(path, seed, batch_size)
        # path: path to the LLVIP dataset
        image_paths = glob(path + "\\visible\\test\\*.jpg", recursive = True)
        thermal_paths = glob(path + "\\infrared\\test\\*.jpg", recursive = True)
        
        batch_idcs = self._get_random_subsample(len(image_paths))
        image_paths = np.array(image_paths)[batch_idcs]
        thermal_paths = np.array(thermal_paths)[batch_idcs]
        
        all_images = []
        for image_path in image_paths:
            with open(image_path, "rb") as fopen:
                image = Image.open(fopen).convert("RGB")
                all_images.append(image)

            
        all_thermals = []
        
        for thermal_path in thermal_paths:
            with open(thermal_path, "rb") as fopen:
                thermal = Image.open(fopen).convert("L")
                all_thermals.append(thermal)

        self.MODE2_Type = ThermalType

        # TODO... load images and thermals on demand with a custom loader
        self.all_images = np.array(all_images)
        self.all_prompts = np.array(all_thermals)


In [ ]:
# download dataset: https://bupt-ai-cz.github.io/LLVIP/
dataset = LLVIP_Dataset(path="C:\\Users\\Christina\\Data\\imagebind\\LLVIP\\", batch_size=100) 
all_images, all_thermals = dataset.get_data()
print(len(all_images), len(all_thermals))

am_widgets.CLIPExplorerWidget("test_thermal", all_data={"image": all_images, "thermal": all_thermals}, models=[am_model.ImageBind_Model()]) 

C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\3221552142.py:99: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  self.all_images = np.array(all_images)
C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\3221552142.py:99: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.all_images = np.array(all_images)
C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\32

100 100
found cached embeddings for test_thermal_ImageBind_huge image
batch 1 of 1
torch.Size([100, 1, 224, 224])


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('ImageBind…

# Image-Depth

In [ ]:
from glob import glob
from PIL import Image
import numpy as np
import torch
import io
import matplotlib.pyplot as plt


class DepthType(am_data.ImageType):
    name = "Depth"

    def __init__(self, data) -> None:
        # data is an array of (1,224,224) tensors
        super().__init__(data)

    def getVisItem(self, idx):
        output_img = io.BytesIO()
        plt.imsave(output_img, self.data[idx][0], cmap='gray')
        plt.savefig(output_img, format='JPEG')
        return {"displayType": am_data.DisplayTypes.IMAGE, "value": output_img.getvalue()}
    
    

class SUNRGBD_Dataset(am_data.DatasetInterface):
    name='SUNRGBD_NYU'

    def __init__(self, path, seed=31415, batch_size = 100):
        # download "SUNRGBD_V1" dataset from https://rgbd.cs.princeton.edu/
        super().__init__(path, seed, batch_size)
        # path: path to the SUNRGBD dataset
        image_paths = glob(path + "\\kv1\\NYUdata\\NYU*\\fullres\\*.jpg", recursive = True)
        depth_paths = glob(path + "\\kv1\\NYUdata\\NYU*\\fullres\\*.png", recursive = True)
        
        batch_idcs = self._get_random_subsample(len(image_paths))
        image_paths = np.array(image_paths)[batch_idcs]
        depth_paths = np.array(depth_paths)[batch_idcs]
        
        all_images = []
        for image_path in image_paths:
            with open(image_path, "rb") as fopen:
                image = Image.open(fopen).convert("RGB")
                all_images.append(image)

            
        all_depths = []
        for depth_path in depth_paths:
            with open(depth_path, "rb") as fopen:
                depth = Image.open(fopen)
                depth = np.array(depth, dtype=int)
                depth = depth.astype(np.float32) / depth.max() # TODO: need to normalize?
                # depth = depth[np.newaxis,:,:] # need 1 channel -> (1,224,224)
                depth = torch.from_numpy(depth).unsqueeze(0) # need 1 channel -> (1,224,224)
                all_depths.append(depth)

        self.MODE2_Type = DepthType

        # TODO... load images and depths on demand with a custom loader
        self.all_images = np.array(all_images)
        self.all_prompts = np.array(all_depths)



In [ ]:
# download "SUNRGBD_V1" dataset from https://rgbd.cs.princeton.edu/
dataset = SUNRGBD_Dataset(path="C:\\Users\\Christina\\Data\\imagebind\\SUNRGBD\\", batch_size=100)
all_images, all_depths = dataset.get_data()
print(len(all_images), len(all_depths))

am_widgets.CLIPExplorerWidget("test_depth", all_data={"image": all_images, "depth": all_depths}, models=[am_model.ImageBind_Model()]) 

C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\3221552142.py:64: FutureWarning:

The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.

C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\3221552142.py:64: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

C:\Users\Christina\AppData\Local\Temp\ipykernel_24244\3221552142.py:65: FutureWarning:

The input object of type 'Tensor' is an array-

100 100
found cached embeddings for test_depth_ImageBind_huge image
found cached embeddings for test_depth_ImageBind_huge depth


CLIPExplorerWidget(children=(VBox(children=(HBox(children=(Dropdown(description='Model: ', options=('ImageBind…